In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset
import torchaudio
from torch.utils.data import DataLoader
import os
import numpy as np
import torch.nn.functional as F

import pandas as pd
from konlpy.tag import Okt
okt = Okt()

In [2]:
df = pd.read_csv("./example_df2.csv")
df2 = pd.read_csv("./input_df.csv")

In [3]:
word_set = []
count = 0
for j in range(len(df2["value"])):
    for i in okt.morphs(df2["value"][j]):
        if i not in word_set:
            word_set.append(i)
            count += 1
    if count%1000==0:
        print(count)
for j in range(len(df2["label"])):
    for i in okt.morphs(df2["label"][j]):
        if i not in word_set:
            word_set.append(i)
    if count%1000==0:
        print(count)

2000
4000
7000
8000
8000
9000
9000
12000
15000
15000
16000
17000
20000
21000
25000
25000
26000
27000
30000
31000
32000
32000
35000
35000
35000
37000
39000
39000
39000
39000
39000
39000
40000
42000
42000
42000
42000
44000
45000
45000
45000
46000
48000
49000
51000
52000
52000
52000
52000
52000
52000
53000
53000
54000
55000
60000
62000
62000
62000
63000
63000
63000
63000
63000
63000
64000
64000
65000
65000
65000
65000
65000
67000
67000
67000
67000
67000
68000
69000
69000
69000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
71000
72000
72000
73000
73000
73000
73000
73000
74000
74000
75000
75000
75000
75000
75000
77000
77000
78000
78000
78000
79000
79000
79000
79000
79000
79000
79000
79000
79000
79000
79000
79000
79000
79000
80000
80000
80000
80000
80000
80000
80000
82000
82000
83000
83000
83000
83000
83000
83000
83000
83000
83000
83000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000
84000

In [4]:
vocab = {word: i+2 for i, word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
# print(vocab)

In [118]:
reverse = dict(map(reversed,vocab.items()))

In [6]:
class NewsCorpus(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        input_sentence = self.df["value"][idx]
        label_sentence = self.df["label"][idx]
        
        token_ids = [vocab[i] for i in okt.morphs(input_sentence)]
        x = torch.zeros(647, dtype=torch.long)
        for i in range(len(token_ids)):
            x[i] = token_ids[i]
            
        label_ids = [vocab[i] for i in okt.morphs(label_sentence)]
        y = torch.zeros(647, dtype=torch.float32)
        for i in range(len(label_ids)):
            y[i] = label_ids[i]
        
        return x,y

In [7]:
int(len(df2)*(7/10)), int(len(df2)*(2/10)), int(len(df2)*(1/10))

(123778, 35365, 17682)

In [28]:
train_df = df2.iloc[:1024,:]
valid_df = df2.iloc[1024:1024+256,:]
test_df = df2.iloc[1280:1280+128,:]

In [132]:
tr_dataset = NewsCorpus(train_df)
tr_dataloader = DataLoader(
    tr_dataset,
    batch_size=1,
    shuffle=True,
    pin_memory=True,
#     num_workers=12,
)

val_dataset = NewsCorpus(valid_df)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    pin_memory=True,
#     num_workers=5,
)

ts_dataset = NewsCorpus(test_df)
ts_dataloader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=False,
#     num_workers=5,
)

In [133]:
x,y = next(iter(tr_dataloader))
print(x.shape, y.shape)
print(x.dtype, y.dtype)

torch.Size([1, 647]) torch.Size([1, 647])
torch.int64 torch.float32


In [31]:
batch_size = 128
torch.randn(batch_size, 647, 512).shape

torch.Size([128, 647, 512])

In [153]:
class GRUModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.wte = nn.Embedding(89162, 647, padding_idx=0)
        self.gru1 = nn.GRU(input_size, hidden_size, batch_first=True)
        self.gru2 = nn.GRU(hidden_size, hidden_size, batch_first=True)  # Additional GRU layer
        self.gru3 = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.gru4 = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(165632, output_size) ## 647 * 256
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.flatten = nn.Flatten()
        
    def forward(self, x):
        h0 = nn.init.xavier_normal_(torch.randn(1, batch_size, self.hidden_size).to(x.device), gain=1.0)
        hidden_state0 = h0.cpu().detach().numpy()
        wte = self.wte(x)
        
        output, hidden1 = self.gru1(wte, (h0))
        hidden_state1 = hidden1.cpu().detach().numpy()
        
        output = self.dropout(self.relu(output))
        
        output, hidden2 = self.gru2(output, hidden1)
        hidden_state2 = hidden2.cpu().detach().numpy()
        
        output = self.dropout(self.relu(output))
        
        output, hidden3 = self.gru3(output, hidden2)
        hidden_state3 = hidden3.cpu().detach().numpy()
        
        output = self.dropout(self.relu(output))
        
        output3, hidden4 = self.gru4(output, hidden3)
        hidden_state4 = hidden4.cpu().detach().numpy()
        
        output = self.dropout(self.relu(output3))
        
        output = self.fc(self.relu(self.flatten(output)))
        
        return output, [hidden_state0, hidden_state1, hidden_state2, hidden_state3, hidden_state4, output3]

In [154]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRUModel(x.shape[1], y.shape[1], 256).to(device)
criterion = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=.1)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.85 ** epoch,
#                                         last_epoch=-1,
                                        verbose=True)

Adjusting learning rate of group 0 to 1.0000e-01.


In [155]:
model

GRUModel(
  (wte): Embedding(89162, 647, padding_idx=0)
  (gru1): GRU(647, 256, batch_first=True)
  (gru2): GRU(256, 256, batch_first=True)
  (gru3): GRU(256, 256, batch_first=True)
  (gru4): GRU(256, 256, batch_first=True)
  (fc): Linear(in_features=165632, out_features=647, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

In [35]:
for i, (x,y) in enumerate(tr_dataloader):
        x,y = x.to(device), y.to(device)
        print(x.shape, y.shape)

torch.Size([128, 647]) torch.Size([128, 647])
torch.Size([128, 647]) torch.Size([128, 647])
torch.Size([128, 647]) torch.Size([128, 647])
torch.Size([128, 647]) torch.Size([128, 647])
torch.Size([128, 647]) torch.Size([128, 647])
torch.Size([128, 647]) torch.Size([128, 647])
torch.Size([128, 647]) torch.Size([128, 647])
torch.Size([128, 647]) torch.Size([128, 647])


In [55]:
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import matplotlib.pyplot as plt

colors = ['#85c2e1', '#89c4e2', '#95cae5', '#99cce6', '#a1d0e8',
        '#b2d9ec', '#baddee', '#c2e1f0', '#eff7fb', '#f9e8e8',
        '#f9e8e8', '#f9d4d4', '#f9bdbd', '#f8a8a8', '#f68f8f',
        '#f47676', '#f45f5f', '#f34343', '#f33b3b', '#f42e2e']

cmap = LinearSegmentedColormap.from_list('Custom', colors, len(colors))

def layer_visualization(hidden_states, text):
    plt.figure(figsize=(7 * len(hidden_states), 4)) # Adjust the figure size
    plt.subplots_adjust(wspace=0.2, hspace=0.5)
    for layer_idx, hidden_state in enumerate(hidden_states):
        plt.subplot(1, len(hidden_states), layer_idx + 1) # Create a subplot for each layer
        plt.title(f'Hidden state values in layer {layer_idx + 1}')
        sns.heatmap(hidden_state[0,:20], cmap=cmap)
#         print([reverse[int(i)] for i in text[:10,:20]])
        plt.axis("off")
    plt.show() # Call `plt.show()` once after creating all subplots

In [105]:
def cstr(s, color='black'):
    if s == ' ':
        return "<text style=color:#000;padding-left:10px;background-color:{}> </text>".format(color, s)
    else:
        return "<text style=color:#000;background-color:{}>{} </text>".format(color, s)

In [106]:
from IPython.display import HTML as html_print
from IPython.display import display


def print_color(t):
    display(html_print(''.join([cstr(ti, color=ci) for ti,ci in t])))

In [ ]:
def sigmoid(x):
    z = 1/(1 + np.exp(-x)) 
    return z

In [107]:
display(html_print(''.join(cstr('나', color='#85c2e1'))))

In [79]:
okt.morphs("나는 밥을 먹었습니다")

['나', '는', '밥', '을', '먹었습니다']

In [111]:
t = [["나", '#85c2e1'],
    ["는", '#f42e2e'],
    ["밥", '#f8a8a8'],
    ["을", '#baddee'],
    ["먹었습니다", '#89c4e2']]


In [112]:
print_color(t)

In [156]:
from time import time
timelist = []

batch_size=1
model.train()

num_epochs = 1024
for epoch in range(num_epochs):
    total_loss = 0.0
    total_correct = 0
    total_step = 0
    start = time()
    if epoch%128 == 0:
        torch.save(model.state_dict(), "./Untitled Folder/Text2Text.pt")
    print(epoch)
    for i, (x,y) in enumerate(tr_dataloader):
        x,y = x.to(device), y.to(device)
#         print(x.shape, y.shape)
        
        outputs, hidden_states = model(x)
#         print(outputs.shape)
#         print(len(hidden_states))
        loss = criterion(outputs, y)
#         print(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_step = i

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == y.argmax(dim=1)).sum().item()
    scheduler.step()
    epoch_loss = total_loss / (batch_size * total_step)
    epoch_acc = total_correct / (batch_size * total_step)
    end = time()
    timelist.append(end-start)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Time Duration : {end - start:.4f}')
#     layer_visualization(hidden_states, y)
    print(y)
    print(hidden_states[5])

0
Adjusting learning rate of group 0 to 8.5000e-02.
Epoch [1/1024], Loss: nan, Time Duration : 101.4830
tensor([[369., 453., 454.,   5.,   2.,  18.,  19.,  20., 455., 183.,   5.,   2.,
          62., 281., 107., 359.,  19., 456., 457., 318.,  74.,  58., 458.,   5.,
         459.,   2.,  16., 460., 461.,   5., 462., 463., 464.,  24.,  86.,  18.,
          19.,  20., 465.,  65.,  62., 466., 466., 177.,  72.,  73.,  14.,  13.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
     

KeyboardInterrupt: 

In [136]:
y.shape

torch.Size([1, 647])

In [135]:
hidden_states[4].shape

(1, 1, 256)

In [138]:
hidden_states[4]

array([[[-0.29340002,  1.        , -1.        , -1.        ,
         -1.        ,  1.        ,  1.        ,  1.        ,
         -1.        , -1.        , -1.        , -1.        ,
         -1.        ,  0.9962113 ,  1.        ,  1.        ,
          1.        , -1.        ,  1.        , -1.        ,
         -1.        , -1.        , -1.        ,  1.        ,
          1.        , -1.        , -1.        ,  1.        ,
         -1.        , -1.        ,  1.        , -1.        ,
          1.        ,  1.        ,  1.        , -1.        ,
         -1.        , -1.        ,  0.23928717, -1.        ,
         -1.        , -1.        , -1.        ,  1.        ,
         -1.        ,  1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        ,  1.        ,
         -1.        ,  1.        , -1.        , -1.        ,
          1.        ,  1.        , -1.        , -1.        ,
          1.        , -1.        , -1.        ,  1.        ,
         -1.        , -1

In [127]:
y.cpu().detach().numpy()

array([[5.880e+02, 2.880e+02, 2.530e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.000e+00, 5.000e+00, 6.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [8.520e+02, 7.770e+02, 3.301e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [9.100e+01, 1.087e+03, 2.660e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.190e+02, 1.568e+03, 1.640e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [4.530e+02, 2.400e+01, 4.900e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]], dtype=float32)

In [129]:
[reverse[i] for i in y[0].cpu().detach().numpy()]

['어',
 '그리고',
 '그때',
 '는',
 '그런',
 '직접',
 '적',
 '인',
 '지원',
 '도',
 '물론',
 '필요하지만',
 '그래도',
 '다',
 '같이',
 '모',
 '여서',
 '예',
 '를',
 '들어',
 '부업',
 '을',
 '한다거나',
 '그런',
 '단순한',
 '작업',
 '을',
 '하면서',
 '그런',
 '소득',
 '을',
 '얻는',
 '방식',
 '그러면은',
 '어르신',
 '분',
 '들',
 '도',
 '적적하지',
 '않으실',
 '거',
 '같고',
 '서로',
 '소통',
 '하면서',
 '치매',
 '예방',
 '이나',
 '그런',
 '우울증',
 '예방',
 '그런',
 '것',
 '도',
 '좋을',
 '거',
 '같아요',
 '.',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk

In [15]:
model = torch.load(model.state_dict(), "./Untitled Folder/Text2Text.pt")
model.eval()

AttributeError: 'collections.OrderedDict' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [ ]:
input_ids = tokenizer.encode("밥은 먹었어?", return_tensors="pt")

In [ ]:
## https://ratsgo.github.io/nlpbook/docs/generation/inference1/

with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty = 1.5,
        no_repeat_ngram_size=3,
        top_k=50, 
        temperature=0.9, ## 1보다 작게 하면 정확한 문장을, 1보다 크게 하면 다양한 문장 생성
        top_p = 0.92,
      )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

In [ ]:
## https://ratsgo.github.io/nlpbook/docs/generation/inference2/#%EC%BD%94%EB%93%9C6-%EC%9D%B8%ED%8D%BC%EB%9F%B0%EC%8A%A4

def inference_fn(
        prompt,
        min_length=10,
        max_length=20,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

In [ ]:
from ratsnlp.nlpbook.qa import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()